In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os

# Vérification du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device utilisé :", device)


In [ ]:
# Dossier racine de ton dataset
root_dir = "dataset"  # dossier créé par icrawler

# Transformations pour l'entrainement (avec augmentation)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Transformations pour la validation / test
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset complet
full_dataset = datasets.ImageFolder(root=root_dir, transform=train_transform)
num_classes = len(full_dataset.classes)
print("Classes :", full_dataset.classes)

# Split train / val
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Remplacer transform pour val_dataset
val_dataset.dataset.transform = val_transform

# DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [ ]:
# Utilisation d'un modèle pré-entraîné ResNet18
model = models.resnet18(pretrained=True)

# Remplacer la dernière couche pour notre nombre de classes
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Criterion et optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
num_epochs = 30  # tu peux augmenter selon besoin

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / train_size
    epoch_acc = running_corrects.double() / train_size
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")


In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Matrice de confusion
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(cm, display_labels=full_dataset.classes)
disp.plot(xticks_rotation=90)
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import functional as F

# Sélection d'une image de validation
example_img, _ = val_dataset[0]
input_tensor = example_img.unsqueeze(0).to(device)

# Grad-CAM setup
gradients = []
activations = []

def forward_hook(module, input, output):
    activations.append(output)

def backward_hook(module, grad_input, grad_output):
    gradients.append(grad_output[0])

# On hook la dernière couche conv (layer4[-1].conv2 pour ResNet18)
target_layer = model.layer4[-1].conv2
target_layer.register_forward_hook(forward_hook)
target_layer.register_backward_hook(backward_hook)

# Forward pass
model.eval()
output = model(input_tensor)
pred_class = output.argmax(dim=1)

# Backward pass
model.zero_grad()
output[:, pred_class].backward()

# Récupération des activations et gradients
grad = gradients[0].cpu().data.numpy()[0]
activation = activations[0].cpu().data.numpy()[0]

# Pondération des cartes d'activation par les gradients moyens
weights = np.mean(grad, axis=(1, 2))
cam = np.zeros(activation.shape[1:], dtype=np.float32)

for i, w in enumerate(weights):
    cam += w * activation[i]

# Normalisation
cam = np.maximum(cam, 0)
cam = cam / cam.max()

# Conversion en image
cam = np.uint8(cam * 255)
cam = np.stack([cam]*3, axis=2)
cam = F.resize(torch.tensor(cam).permute(2,0,1), [224,224])

# Affichage
plt.imshow(example_img.permute(1,2,0).cpu())  # image originale
plt.imshow(cam.permute(1,2,0), cmap='jet', alpha=0.5)  # superposé
plt.title(f"Grad-CAM pour la classe: {full_dataset.classes[pred_class]}")
plt.axis("off")
plt.show()
